In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
from geodatasets import get_path
from function_06 import load_data_with_delimiters
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap


In [ ]:
# Read all datas
file_info = "../data/raw/ongevallen.txt"
df_accidents =pd.read_csv(file_info, low_memory=False)

file_info2 = "../data/raw/partijen.txt"
df_A=pd.read_csv(file_info2, low_memory=False)

file_info3 = "../data/raw/wegvakgeografie_01012024.csv"
wegvak_df=pd.read_csv(file_info3, low_memory=False)

In [ ]:
df_accidents.shape, df_accidents.isnull().sum(), df_accidents.columns

In [ ]:
df_accidents['WEEKNR'].unique()

In [ ]:
relevant_columns = [
    'VKL_NUMMER', 'TIJDSTIP', 'UUR', 'JAAR_VKL',
    'ANTL_DOD', 'ANTL_GZH', 'ANTL_SEH',
    'WVK_ID', 'GME_NAAM', 'PLT_NAAM',
    'BZD_VM_AN', 'BZD_IF_AN', 'BZD_TA_AN'
]

# Keep only those columns
df_accidents = df_accidents[relevant_columns]

# Drop rows where critical identifiers or keys are missing
df_accidents = df_accidents.dropna(subset=['VKL_NUMMER', 'WVK_ID'])


In [ ]:
 df_accidents.shape

In [ ]:
df_A.head()

In [ ]:
df_A['PTJ_ID'].head()

In [ ]:
df_A.isnull().sum()

In [ ]:
# df_A = df_A.dropna(subset=['TDT_AN'])
# df_A = df_A.dropna(subset=['OTE_AN'] )

In [ ]:
bike_accidents['OTE_AN'].unique()

In [ ]:
bike_accidents['TDT_AN'].unique()


In [ ]:
# Keywords related to bikes in OTE_AN
bike_keywords_ote = [
    'FIETS', 'E-BIKE', 'SPEED PEDELEC', 'SPEEDPEDELEC', 'FIETSKAR',
    'FIETSAANHANGER', 'DRIEWIELER', 'HANDBIKE', 'FATBIKE', 'FAT BIKE',
    'SKELTER', 'BUGGY', 'ELEKTRISCHE-STEP', 'ELEKTRISCHE STEP', 'ELEKTRISCHE DRIEWIELER'
]

# Keywords indicating cyclist-related circumstances in TDT_AN
bike_keywords_tdt = [
    'FIETS', 'FIETSPAD', 'EVENWICHT', 'PORTIER OPENEN', 'GEEN VERLICHTING',
    'OBSTAKEL', 'SLECHT WEGDEK', 'AUTO OP FIETSPAD', 'ONVOORZICHTIG OP WEG'
]


bike_accidents = df_A[(df_A['OTE_AN'].isin(bike_keywords_ote)) | (df_A['TDT_AN'].isin(bike_keywords_tdt))]

# Filter rows where either column contains one of the keywords (case-insensitive)
#bike_accidents = df_A[
#    df_A['OTE_AN'].str.contains('|'.join(bike_keywords_ote), case=False, na=False) |
#    df_A['TDT_AN'].str.contains('|'.join(bike_keywords_tdt), case=False, na=False)
#]


In [ ]:
bike_accidents = bike_accidents[['PTJ_ID', 'VKL_NUMMER', 'NUMMER','OTE_AN','TDT_AN']]


In [ ]:
bike_accidents.head()

In [ ]:
bike_accidents.isnull().sum()

In [ ]:
# Filter the wegvak dataset for rows where the municipality is Amsterdam
wegvak_amsterdam = wegvak_df[wegvak_df['gme_naam'].str.lower() == 'amsterdam']

# Check how many rows and preview a few
wegvak_amsterdam.shape, wegvak_amsterdam[['wvk_id', 'gme_naam', 'shape']].head()


In [ ]:
columns_to_keep = [
    'wvk_id',           # Road Segment ID
    'wvk_begdat',       # Start Date
    'wvk_enddat',       # End Date (optional if not always available)
    'wegnummer',        # Road Number
    'stt_naam',         # Street Name
    'wpsnaam',          # Neighborhood Name
    'gme_naam',         # Municipality Name
    'rijrichtng',       # Driving Direction
    'wegbehsrt',        # Road Type / Category
    'shape'             # shape
]


In [ ]:
wegvak_amsterdam = wegvak_amsterdam[columns_to_keep]

# Drop rows with NaN in critical columns
#wegvak_amsterdam = wegvak_amsterdam.dropna(subset=['wvk_id', 'wegnummer', 'stt_naam', 'shape'])


In [ ]:
# Ensure column name consistency for the join
bike_data_geo = df_accidents.copy()

# Confirm WVK_ID exists in both datasets
if 'WVK_ID' in bike_data_geo.columns and 'wvk_id' in wegvak_amsterdam.columns:
    # Merge on WVK_ID
    bike_amsterdam_geo = bike_data_geo.merge(
        wegvak_amsterdam,
        left_on='WVK_ID',
        right_on='wvk_id',
        how='inner'
    )
    match_found = True
else:
    bike_amsterdam_geo = pd.DataFrame()
    match_found = False

# Output results
# match_found, bike_amsterdam_geo[['Accident_ID', 'Vehicle_Type', 'Municipality', 'wvk_id', 'shape']].head() if match_found else "No matches found"


In [ ]:
bike_amsterdam_geo['shape'].head()


In [ ]:
bike_accidents.head()

In [ ]:
bike_data = bike_accidents.merge(bike_amsterdam_geo, on='VKL_NUMMER', how='outer')


In [ ]:
bike_data.shape

In [ ]:
bike_data.isnull().sum()

In [ ]:
# Drop 'WVK_ID' (duplicate of 'wvk_id') and 'Unnamed: 0' (auto index column)
#bike_data = bike_data.drop(columns=['WVK_ID'])
rename_dict = {
    'PTJ_ID': 'Participant_ID',
    'VKL_NUMMER': 'Accident_ID',
    'NUMMER': 'Case_Number',
    'OTE_AN': 'Object_Involved',
    'TDT_AN': 'Contributing_Factor',
    'TIJDSTIP': 'Time',
    'UUR': 'Hour',
    'JAAR_VKL': 'Accident_Year',
    'ANTL_DOD': 'Fatalities',
    'ANTL_GZH': 'Severe_Injuries',
    'ANTL_SEH': 'Emergency_Room_Injuries',
    'WVK_ID': 'RoadSegment_ID',
    'GME_NAAM': 'Municipality',
    'PLT_NAAM': 'Police_Area',
    'BZD_VM_AN': 'Vehicle_Type',
    'BZD_IF_AN': 'Object_Collided_With',
    'BZD_TA_AN': 'Other_Party_Involved',
    'wvk_id': 'RoadSegment_ID',  # duplicate
    'wvk_begdat': 'Segment_StartDate',
    'wvk_enddat': 'Segment_EndDate',
    'wegnummer': 'RoadNumber',
    'stt_naam': 'StreetName',
    'wpsnaam': 'Neighborhood',
    'gme_naam': 'Municipality',  # duplicate
    'rijrichtng': 'DrivingDirection',
    'wegbehsrt': 'RoadType',
    'shape': 'Shape'
}

# Step 3: Optional rename for clarity (if needed)
bike_data = bike_data.rename(columns=rename_dict)

In [ ]:
# Display updated column names
bike_data.columns.tolist()

In [ ]:
bike_data.columns = bike_data.columns.str.lower()


In [ ]:
bike_data.isnull().sum()

In [ ]:
df_ams_cc = bike_data.drop(columns=['contributing_factor','roadtype', 'drivingdirection', 'case_number','object_involved','police_area', 'time', 'hour','other_party_involved','object_collided_with','emergency_room_injuries','severe_injuries','fatalities', 'vehicle_type'], errors='ignore')


In [ ]:
df_ams_cc.to_csv('../data/cleaned/df_ams_cc.csv')

In [ ]:
bike_data.to_csv('../data/cleaned/df_ams_ccenter.csv')